In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime as dt
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from uszipcode import SearchEngine
%matplotlib inline
plt.style.use('seaborn')

In [4]:
df = pd.read_csv('df_cleaned.csv')
df.head()

,date_sold,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,sqft_above,yr_built,...,lat,long,sqft_living15,sqft_lot15,basement,renovated,condition,grade,city,location
0,735519,221900.0,3,1,1180,5650,1,0,1180,1955,...,47.5112,-122.257,1340,5650,0,0,2,4,Seattle,SW
1,735576,538000.0,3,2,2570,7242,2,0,2170,1951,...,47.7210,-122.319,1690,7639,1,1,2,4,Seattle,NW
2,735654,180000.0,2,1,770,10000,1,0,770,1933,...,47.7379,-122.233,2720,8062,0,0,2,3,Kenmore,NE
3,735576,604000.0,4,3,1960,5000,1,0,1050,1965,...,47.5208,-122.393,1360,5000,1,0,4,4,Seattle,NW
4,735647,510000.0,3,2,1680,8080,1,0,1680,1987,...,47.6168,-122.045,1800,7503,0,0,2,5,Sammamish,NE


#### Final Call on Variables

* Dependent Variable (target):
  - Price
 
 
* Independent Variables:
  - **Continuous**: ['sqft_living','yr_built','sqft_above','waterfront','condition','floors','bedrooms','basement','renovated','bathrooms', 'grade']




  - **Drop**: ['sqft_lot', 'sqft_lot15','date_sold','zipcode','lat','long']
  
     - **sqft_lot** and **sqft_lot15**: there is no linear relationship with target variable
     - **zipcode**: by using zipcode we added the **city** column
     - **lat** and **long**: by using them a column named **location** added
     - **date_sold**: we have all kind of prices in each date of a year 
     
     

   - **Create dummies**: ['location', 'city']
  
  

   - **Binary Variables**: ['renovated','basement','waterfront']

  

In [7]:
categoricals = ['city','location']

continuous = ['sqft_living''yr_built','sqft_above','waterfront','condition','floors','bedrooms'
              ,'sqft_living15','grade','basement','renovated','bathrooms','grade']

deleted = ['sqft_lot15','sqft_lot','date_sold','zipcode','lat','long']

In [8]:
#creating our final DataFrame after doing some research on categorical and continuous variables 
dummies = df[categoricals].astype('category')
dummies = pd.get_dummies(dummies,prefix=dummies.columns, drop_first=True)
df_preprocessed = df.drop(categoricals, axis=1)
df_preprocessed = pd.concat([df_preprocessed, dummies], axis=1)
df_preprocessed = df_preprocessed.drop(deleted, axis=1)
df_fin_cat = df_preprocessed.copy()
df_fin_cat.head()

,price,bedrooms,bathrooms,sqft_living,floors,waterfront,sqft_above,yr_built,sqft_living15,basement,...,city_Redmond,city_Renton,city_Sammamish,city_Seattle,city_Snoqualmie,city_Vashon,city_Woodinville,location_NW,location_SE,location_SW
0,221900.0,3,1,1180,1,0,1180,1955,1340,0,...,0,0,0,1,0,0,0,0,0,1
1,538000.0,3,2,2570,2,0,2170,1951,1690,1,...,0,0,0,1,0,0,0,1,0,0
2,180000.0,2,1,770,1,0,770,1933,2720,0,...,0,0,0,0,0,0,0,0,0,0
3,604000.0,4,3,1960,1,0,1050,1965,1360,1,...,0,0,0,1,0,0,0,1,0,0
4,510000.0,3,2,1680,1,0,1680,1987,1800,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
df_fin_cat.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'floors', 'waterfront',
       'sqft_above', 'yr_built', 'sqft_living15', 'basement', 'renovated',
       'condition', 'grade', 'city_Bellevue', 'city_Black Diamond',
       'city_Bothell', 'city_Carnation', 'city_Duvall', 'city_Enumclaw',
       'city_Fall City', 'city_Federal Way', 'city_Issaquah', 'city_Kenmore',
       'city_Kent', 'city_Kirkland', 'city_Maple Valley', 'city_Medina',
       'city_Mercer Island', 'city_North Bend', 'city_Redmond', 'city_Renton',
       'city_Sammamish', 'city_Seattle', 'city_Snoqualmie', 'city_Vashon',
       'city_Woodinville', 'location_NW', 'location_SE', 'location_SW'],
      dtype='object')

#### Address Multicolinearity Before Train-Test_Split

In [10]:
#create a dataframe to address multicolinearity
df_fin_mcl = df_fin_cat.corr().abs().stack().reset_index().sort_values(0,ascending=False)
df_fin_mcl['pairs'] = list(zip(df_fin_mcl.level_0,df_fin_mcl.level_1))
df_fin_mcl.set_index(['pairs'],inplace=True)
df_fin_mcl.drop(columns=['level_0','level_1'],inplace=True)
df_fin_mcl.columns = ['Correlation']
df_fin_mcl.drop_duplicates(inplace=True)
df_fin_mcl[(df_fin_mcl.Correlation>.70)&(df_fin_mcl.Correlation<1)]

,Correlation
pairs,
"(sqft_living, sqft_above)",0.876448
"(grade, sqft_living)",0.762779
"(sqft_living, sqft_living15)",0.756402
"(sqft_above, grade)",0.756073
"(sqft_living15, sqft_above)",0.731767
"(sqft_living, bathrooms)",0.720891
"(grade, sqft_living15)",0.713867
"(price, sqft_living)",0.701917


* There is a high multicolinearity between squarefoot living and squarefoot above, so we decided to drop squarefeet above to reduce multicolinearity in our dataset.

In [11]:
df_fin_cat.drop('sqft_above', axis=1, inplace=True)

In [12]:
df_fin_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 38 columns):
price                 21597 non-null float64
bedrooms              21597 non-null int64
bathrooms             21597 non-null int64
sqft_living           21597 non-null int64
floors                21597 non-null int64
waterfront            21597 non-null int64
yr_built              21597 non-null int64
sqft_living15         21597 non-null int64
basement              21597 non-null int64
renovated             21597 non-null int64
condition             21597 non-null int64
grade                 21597 non-null int64
city_Bellevue         21597 non-null uint8
city_Black Diamond    21597 non-null uint8
city_Bothell          21597 non-null uint8
city_Carnation        21597 non-null uint8
city_Duvall           21597 non-null uint8
city_Enumclaw         21597 non-null uint8
city_Fall City        21597 non-null uint8
city_Federal Way      21597 non-null uint8
city_Issaquah         21597 n

#### Train-Test-Split 

* We use TTS to create our basemodel and continue to train our model on train set

In [14]:
target = 'price'
x_cols = list(df_fin_cat.columns)
x_cols.remove(target)

In [15]:
x = df_fin_cat[x_cols]
y = df_fin_cat[['price']]


In [16]:
# Our First TTS, test size is 20% of our data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=52)
print(len(x_train), len(y_test))
x_train.head()

17277 4320


,bedrooms,bathrooms,sqft_living,floors,waterfront,yr_built,sqft_living15,basement,renovated,condition,...,city_Redmond,city_Renton,city_Sammamish,city_Seattle,city_Snoqualmie,city_Vashon,city_Woodinville,location_NW,location_SE,location_SW
15476,2,1,1210,1,0,1952,1210,0,0,2,...,0,0,0,1,0,0,0,0,0,0
13427,3,2,2120,2,0,1981,2620,0,0,3,...,0,0,0,0,0,0,0,0,0,0
20387,3,2,1270,2,0,2012,1490,0,0,2,...,0,0,0,1,0,0,0,0,0,0
4244,4,3,2680,2,0,2009,2750,0,0,2,...,1,0,0,0,0,0,0,0,0,0
12520,3,2,1380,1,0,1915,1250,1,0,3,...,0,0,0,1,0,0,0,1,0,0


because I decided to use sm.ols and then .fit, instead of ols().fit, constants need to be calculated manually for our intercept

In [ ]:
x_train